## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import warnings
from pymystem3 import Mystem
warnings.filterwarnings('ignore')
data_table = pd.read_csv('/datasets/data.csv')
data_table

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

    В представленной таблице имеются пропуски в столбцах 'days_employed' и 'total_income' представляющие из себя как общий трудовой стаж клиента и доход в месяц соответственно. Данные пропущенные значения могли возникнуть по многим причинам. Если взять столбец 'days_employed' представляющий трудовой стаж клиента, то пропуски могли возникнуть как по технологической причине (ошибка в выгрузке данных), так и в связи с человеческим фактором: 
1.Человек может нигде не работать. 
2. Человек при заполнении анкеты, мог не указать информацию по данному вопросу, либо не подтвердить данную информацию сопутствующими документами при оформлении кредита. 
    В любом случае для дальнейшей работы с данной таблицей и более точного исследования нам необходимо избавиться от данных пропусков.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data_table['days_employed'].isnull().sum()

2174

In [3]:
data_table['total_income'].isnull().sum()

2174

In [4]:
total_income_median = data_table['total_income'].median()
data_table['total_income'] = data_table['total_income'].fillna(total_income_median)

In [5]:
data_table['total_income'].isnull().sum()

0

In [6]:
data_table['days_employed'] = data_table['days_employed'].fillna(0)
data_table['days_employed'].isnull().sum()

0

In [7]:
data_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

После обнаружения пропущенных значений, заполняем их по следующему принципу:
1. В столбце 'total_income' определяем медиану и заполняем пропущенные значения полученным значением.
2. В столбце 'days_employed' так как не ясна причина пропусков и в последующих исследованиях данные значения нам не нужны можем можем заменить их нулевым значением.

### Замена типа данных

In [8]:
data_table['days_employed'] = abs(data_table['days_employed'])
data_table['children'] = abs(data_table['children'])
data_table['education'] = data_table['education'].str.lower()
data_table['family_status'] = data_table['family_status'].str.lower()
data_table['income_type'] = data_table['income_type'].str.lower()
data_table['purpose'] = data_table['purpose'].str.lower()

In [9]:
data_table['days_employed'] = data_table['days_employed'].astype('int')
data_table['total_income'] = data_table['total_income'].astype('int')
data_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

При работе по замене типов данных, применим метод astype() с аргументом 'int', с целью замены столбцов с типом данных float64 на int64. Так же методом abs() с целью избавления от отрицательных значений 'days_employed', 'children'. Так же для более точного исследования и точного поиска дупликатов приведем к нижнему регистру методом str.lower() в столбцах 'education', 'family_status', 'income_type', 'purpose'

### Обработка дубликатов

In [10]:
data_table.duplicated().sum()

71

In [11]:
data_table = data_table.drop_duplicates().reset_index(drop = True)

In [12]:
data_table.duplicated().sum()

0

### Вывод

Обработку будем проводить методом duplicated() и выровним по индексу с помощью метода reset_index().

### Лемматизация

In [13]:
m = Mystem()
lemma = data_table['purpose'].apply(m.lemmatize)
lemma.value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

### Вывод

С целью приведения к общим категориям проведем лемматизацию столбца 'purpose' с помощью метода m.lemmatize которую предварительно импортировали из библиотеки pymystem3.

### Категоризация данных

In [14]:
def filtr_lem_func(lemma):
    if 'свадьба' in lemma:
        return 'свадьба'
    if 'автомобиль' in lemma:
        return 'автомобиль'
    if 'образование' in lemma:
        return 'образование'
    if 'недвижимость' or 'жилье' in lemma:
        return 'недвижимость'
    if 'ремонт' in lemma:
        return 'ремонт'
    return 'прочие'
data_table['category_credit'] = lemma.apply(filtr_lem_func)
data_table.head(50)
        

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_credit
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [15]:
data_table['category_credit'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: category_credit, dtype: int64

### Вывод

В данном cell используем метод value_counts() для посчета количества выданных кредитов по каждой категории. Из полученных данных мы видим что большинство кредитов оформляется на покупку, либо ремонт недвижимости, дальше идут на приобретение автомобиля и образования, и на последнем месте для проведения свадьбы.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
part_debt_data_table = data_table.groupby('children').agg({'purpose': 'count', 'debt': 'mean'}).sort_values(by = 'debt')
part_debt_data_table

,purpose,debt
children,,
5,9,0.000000
0,14091,0.075438
3,330,0.081818
1,4855,0.091658
2,2052,0.094542
4,41,0.097561
20,76,0.105263


### Вывод

Сгрупируем данные по методом groupby() по количеству детей и методом agg() сделаем выборку по количеству выданных кредитов и их возврату применив к столбцу 'debt' метод mean().Из данного исследования можем сделать вывод что доля возврата по кредиту у клиента без детей выше, чем у клиента с их наличием. В связи с чем можно порекомендовать выдачу кредитов клиентам без детей. Или рассматривать данный факт как положительный фактор. (Строку с количеством 5 детей считаем не целесообразным учитывать в связи с малым количеством выданных кредитов данной группе людей.)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
data_table.groupby('family_status').agg({'purpose': 'count', 'debt': 'mean'}).sort_values(by = 'debt')

,purpose,debt
family_status,,
вдовец / вдова,959,0.065693
в разводе,1195,0.071130
женат / замужем,12339,0.075452
гражданский брак,4151,0.093471
не женат / не замужем,2810,0.097509


### Вывод

В определении зависимости между семейным положением и возвратом кредита в срок сгруппируем данные по столбцу 'family_status' и методом agg() выведем количество выданных кредитов по столбцу 'purpose' и доля возврата его в срок по столбцу 'debt'  методом mean(). Из полученных данных мы видим что количество возвратом кредитов в срок на первом месте находится категория граждан 'вдовец / вдова', на втором месте люди 'в разводе'. Так же мы видим что самыми ненадежными заемщиками являются люди в группе 'не женат / не замужем' и 'гражданский брак' 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [18]:
data_table_income = [['total_income', 'category_credit', 'debt']]
data_table['income_category'] = pd.qcut(data_table['total_income'], 4)
data_table.groupby('income_category').agg({'purpose': 'count', 'debt': 'mean'}).sort_values(by = 'income_category')

,purpose,debt
income_category,,
"(20666.999, 107623.0]",5364,0.079605
"(107623.0, 145017.0]",6415,0.085269
"(145017.0, 195813.25]",4311,0.089074
"(195813.25, 2265604.0]",5364,0.071402


### Вывод

Перед решением следующего поставленного нам вопроса создадим новый столбец 'income_category' где приведем к категориям по уровню ежемесячного дохода методом qcut(), а в качестве аргументов используем столбец 'total_income' и количества категорий на выходе. Далее сгруппируем по столбцу 'income_category' и посчитаем количество выданных кредитов и долю возврата по каждой категории. Из полученных данных мы видим что самая большая доля возвратов приходится на людей с высоким ежемесячным доходом, а конкретно в промежутке (195813.25, 2265604.0), на втором месте по доле возвратов находятся на оборот, люди с самым низким ежемесячным доходом в промежутке (20666.999, 107623.0)	

- Как разные цели кредита влияют на его возврат в срок?

In [19]:
data_table.groupby('category_credit').agg({'purpose': 'count', 'debt': 'mean'}).sort_values(by = 'debt')

,purpose,debt
category_credit,,
недвижимость,10811,0.072334
свадьба,2324,0.080034
образование,4013,0.092200
автомобиль,4306,0.093590


### Вывод

При ислледовании вопроса как цели кредита влияют на его возврат в срок, было установлено что самая высокая доля возврата зафиксирована в категории связанной с недвижимостью, а самая низкая доля в категории автомобиль.

### Шаг 4. Общий вывод

При исследовании надёжности заёмщиков можем сделать выводы что самым надежным заемщиком является человек, с высоким ежемесячным доходом либо как это неудивительно звучит с самым низким ежемесячным доходом, женатого либо замужнего или овдовевшего, не имеющего детей и оформляющий кредит на цели связанные с недвижимостью.